In [1]:
import sqlite3
import pymongo
import pandas as pd
import requests
import json

In [3]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [4]:
keys = get_keys("/Users/wvsharber/.secret/darksky_api.json")
api_key = keys['api_key']

In [5]:
def getrain(div, date):
    if div == 'E0':
        coor = (51.5074, -0.1278) #london
    else:
        coor = (52.5200, 13.4050) #berlin
    iso_date_time = f'{date}T12:00:00'
    url_template = "https://api.darksky.net/forecast/{}/{},{},{}?exclude=currently,minutely,hourly,alerts,flags"
    request_url = url_template.format(
                                        api_key,
                                        coor[0], #latitude
                                        coor[1], #longitude
                                        iso_date_time)
    response = requests.get(request_url)
    dictapi = response.json()
    weather = dictapi['daily']['data'][0].get('precipType')
    if weather == 'rain':
        return True
    else:
        return False


In [213]:
def pullmatchdata(season):
    conn = sqlite3.connect("database.sqlite")
    search_home = f"""SELECT HomeTeam AS Team, Season, Date, Div, FTHG AS Goals, FTR,
                      CASE 
                          WHEN FTR == 'H' THEN 'win'
                          WHEN FTR == 'A' THEN 'loss'
                          ELSE 'draw'
                      END result
                      FROM matches
                      WHERE Season = {season} AND (Div = 'D1' OR Div = 'E0')
                      ORDER BY HomeTeam;"""
    dfhome = pd.read_sql_query(search_home, conn)
    search_away = f"""SELECT AwayTeam AS Team, Season, Date, Div, FTAG AS Goals, FTR,
                          CASE 
                          WHEN FTR == 'A' THEN 'win'
                          WHEN FTR == 'H' THEN 'loss'
                          ELSE 'draw'
                      END result
                      FROM matches
                      WHERE Season = {season} AND (Div = 'D1' OR Div = 'E0')
                      ORDER BY AwayTeam;"""
    dfaway = pd.read_sql_query(search_away, conn)
    df = pd.concat([dfhome, dfaway], ignore_index = True)
    
    dfE0 = df[df['Div'] == 'E0']['Date'].unique()
    dfD1 = df[df['Div'] == 'D1']['Date'].unique()
    
    dfE0_new = df[df['Div'] == 'E0']
    dfD1_new = df[df['Div'] == 'D1']

#     #pulling information
    dictE0 = {}
    dictD1 = {}
    test_string = ""
    num = 0
    for each in dfE0:
        if num%2==0:
            dictE0[each] = True #getrain(div = 'E0', date = each)
            num+=1
        else:
            dictE0[each] = False
            num+= 1
    for each in dfD1:
        if num%2==0:
            dictD1[each] = True #getrain(div = 'D1', date = each)
            num+=1
        else:
            dictD1[each] = False
            num+= 1
    
    
    dataE0 ={'Date': list(dictE0.keys()), 'Rain': list(dictE0.values())}
    dataD1 ={'Date': list(dictD1.keys()), 'Rain': list(dictD1.values())}
             
    df_rain_E0 = pd.DataFrame(data = dataE0)
    df_E0 = pd.merge(dfE0_new, df_rain_E0, on ='Date', how = 'left')
    
    df_rain_D1 = pd.DataFrame(data = dataD1)
    df_D1 = pd.merge(dfD1_new, df_rain_D1, on ='Date', how = 'left')
    
    final_df = pd.concat([df_E0, df_D1], ignore_index = True)
    
    return final_df



In [214]:
work_df = pullmatchdata(2010)

In [215]:
work_df

,Team,Season,Date,Div,Goals,FTR,result,Rain
0,Arsenal,2010,2010-08-21,E0,6,H,win,True
1,Arsenal,2010,2010-09-11,E0,4,H,win,False
2,Arsenal,2010,2010-09-25,E0,2,A,loss,True
3,Arsenal,2010,2010-10-16,E0,2,H,win,False
4,Arsenal,2010,2010-10-30,E0,1,H,win,True
5,Arsenal,2010,2010-11-07,E0,0,A,loss,False
6,Arsenal,2010,2010-11-20,E0,2,A,loss,True
7,Arsenal,2010,2010-12-04,E0,2,H,win,False
8,Arsenal,2010,2010-12-27,E0,3,H,win,True
9,Arsenal,2010,2011-01-05,E0,0,D,draw,False


In [179]:
work_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 8 columns):
Team      1372 non-null object
Season    1372 non-null int64
Date      1372 non-null object
Div       1372 non-null object
Goals     1372 non-null int64
FTR       1372 non-null object
result    1372 non-null object
Rain      1372 non-null bool
dtypes: bool(1), int64(2), object(5)
memory usage: 76.4+ KB


In [231]:
def createrecord(df_test):
    record = pd.DataFrame()
    record['name'] = df_test['Team'].unique()
    df = df_test.set_index('Team')
    df_rain = df[df.Rain == True]
    for name in record:
        record['league'] = record['name'].map(lambda x: list(df.loc[x,'Div'].unique())[0])
    record['total_goals'] = list(df['Goals'].groupby('Team').sum())
    record['total_wins'] = list(df[df['result'] == 'win'].groupby('Team').count()['result'])
    record['rain_wins'] = list(df_rain[df_rain['result'] == 'win'].groupby('Team').count()['result'])
    record['rain_games'] = list(df_rain.groupby('Team').count()['result'])
    record['rain_percentage'] = round(record['rain_wins']/record['rain_games']*100)
    del record['rain_wins'], record['rain_games']
    return record
    

In [232]:
createrecord(work_df)

,name,league,total_goals,total_wins,rain_percentage
0,Arsenal,E0,72,19,41.0
1,Aston Villa,E0,48,12,35.0
2,Birmingham,E0,81,19,63.0
3,Blackburn,E0,37,8,26.0
4,Blackpool,E0,46,11,21.0
5,Bolton,E0,55,10,10.0
6,Chelsea,E0,52,12,47.0
7,Everton,E0,69,21,45.0
8,Fulham,E0,67,23,67.0
9,Liverpool,E0,31,9,25.0
